# Imports

In [1]:
!pip install safetensors --quiet
!pip install wandb --quiet 
!pip install optuna --quiet

In [2]:
!pip install pandas --quiet
!pip install matplotlib --quiet
!pip install transformers --quiet
!pip install scikit-learn --quiet
!pip install pyarrow --quiet
!pip install transformers[torch] --quiet
!pip install accelerate --quiet

In [1]:
import wandb
wandb.login()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shynkarov-pn (shynkarov-pn-ukrainian-catholic-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
wandb.init(
    project="ukrainian-sentiment",  # Name your project
    name="roberta-ukrainian-sentiment",  # Optional run name
    tags=["roberta", "ukrainian", "sentiment"],  # Optional tags for filtering
)


In [12]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import optuna

from transformers import pipeline, BertConfig, BertForSequenceClassification, BertTokenizer
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.utils.class_weight import compute_class_weight

# Data

In [5]:
df = pd.read_parquet('df_augmented_full_final.parquet')

In [6]:
df

,stratification_label,document_content,language,username,annotator_sentiment,user_id,document_id,document_length,is_ck_annotation,unique_document_id,df_set
0,positive_ua,"Треба максимальна підтримка, прошу поширення",ua,D,positive,467130971,3347,44,0,3347_0,validation
1,neutral_ua,Чому на замовлення ватних дисків сьогодні не з...,ua,O,neutral,277133851,1117,68,0,1117_0,validation
2,positive_ru,"""Прекрасное настроение наполняет сердце радост...",ru,gpt,positive,-1,23500,101,0,23500_0,validation
3,positive_mixed,✨ Отримала подарункові сертифікати на будівель...,mixed,gpt,positive,-1,35734,530,0,35734_0,validation
4,negative_ua,"Трохи несправедливо відносяться до молоді. ""Зн...",ua,gpt,negative,-1,13567,200,0,13567_0,validation
...,...,...,...,...,...,...,...,...,...,...,...
39487,negative_ru,"Дом напротив муз.училища.Фото старое,дома уже нет",ru,D,negative,467130971,10308,49,0,10308_0,test
39488,negative_ua,Розумію так. За неявку без поважної причини по...,ua,D,negative,467130971,5930,998,0,5930_0,test
39489,neutral_ua,фізичне та ментальне відновлення 3️⃣Економічна...,ua,D,neutral,467130971,11410,461,0,11410_0,test
39490,negative_ua,Німеччина передасть Україні сучасні ППО та рад...,ua,D,negative,467130971,2967,266,0,2967_0,test


In [6]:
# df = df.loc[df['annotator_sentiment'] != 'mixed']

In [7]:
df.shape

(39492, 11)

In [8]:
splits_df = {}

for sett in df.df_set.unique():
    splits_df[sett] = df.loc[df['df_set'] == sett].copy()

In [9]:
train_df = splits_df['train']
val_df = splits_df['validation']
test_df = splits_df['test']

In [ ]:
# train_df = train_df.loc[:, ['document_content', 'annotator_sentiment']]

# Model

In [10]:
num_labels=df.annotator_sentiment.nunique()

In [11]:
num_labels

4

In [13]:
config = BertConfig.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=num_labels,
    hidden_dropout_prob=0.2,    # Increase from default (typically 0.1)
    attention_probs_dropout_prob=0.2
)

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [14]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", config=config)
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [15]:
tokenizer("Hello world")['input_ids']

[0, 44, 7802, 83, 4605, 14826, 2]

In [16]:
len(tokenizer.tokenize("Hello world"))

5

In [17]:
token_lengths = []
for text in df['document_content']:
    tokens = tokenizer.tokenize(text)
    token_lengths.append(len(tokens))

print(f"Average tokens per document: {np.mean(token_lengths)}")
print(f"Median tokens per document: {np.median(token_lengths)}")
print(f"Max tokens per document: {np.max(token_lengths)}")
print(f"Documents exceeding 512 tokens: {sum(np.array(token_lengths) > 512)}")

Average tokens per document: 47.95983996758837
Median tokens per document: 35.0
Max tokens per document: 1894
Documents exceeding 512 tokens: 40


In [18]:
len(token_lengths)

39492

In [20]:
# df['token_lengths'] = token_lengths

In [21]:
# df.loc[df.document_length > 1000]

# Training inputs

In [15]:
# Define maximum sequence length (check max length for your specific model)
MAX_LENGTH = 512

In [16]:
# Function to create data loaders
def create_data_loaders(train_dataset, val_dataset, test_dataset, batch_size=16):
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False
    )

    return train_loader, val_loader, test_loader

In [17]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512, strategy="truncate"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.strategy = strategy

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # Different strategies for handling long texts
        if self.strategy == "truncate":
            # Simple truncation from the beginning
            encoding = self.tokenizer(
                text,
                truncation=True,
                padding='max_length',
                max_length=self.max_length,
                return_tensors='pt'
            )

        elif self.strategy == "head_tail":
            # Take first half tokens from beginning, second half from end
            tokens = self.tokenizer.tokenize(text)
            if len(tokens) > self.max_length - 2:  # Account for special tokens
                half_length = (self.max_length - 2) // 2
                tokens = tokens[:half_length] + tokens[-half_length:]

            encoding = self.tokenizer.encode_plus(
                self.tokenizer.convert_tokens_to_string(tokens),
                truncation=True,
                padding='max_length',
                max_length=self.max_length,
                return_tensors='pt'
            )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [18]:
# Function to process dataset with chosen strategy
def prepare_datasets(train_df, val_df, test_df, tokenizer, max_length=512, strategy="truncate"):
    # Encode the sentiment labels
    label_encoder = LabelEncoder()

    # Fit on the entire dataset to ensure all classes are included
    all_sentiments = pd.concat([
        train_df['annotator_sentiment'],
        val_df['annotator_sentiment'],
        test_df['annotator_sentiment']
    ])
    label_encoder.fit(all_sentiments)

    # Transform the labels
    train_labels = label_encoder.transform(train_df['annotator_sentiment'])
    val_labels = label_encoder.transform(val_df['annotator_sentiment'])
    test_labels = label_encoder.transform(test_df['annotator_sentiment'])

    # Create datasets
    train_dataset = SentimentDataset(
        train_df['document_content'].values,
        train_labels,
        tokenizer,
        max_length,
        strategy
    )

    val_dataset = SentimentDataset(
        val_df['document_content'].values,
        val_labels,
        tokenizer,
        max_length,
        strategy
    )

    test_dataset = SentimentDataset(
        test_df['document_content'].values,
        test_labels,
        tokenizer,
        max_length,
        strategy
    )

    return train_dataset, val_dataset, test_dataset, label_encoder

In [19]:
train_dataset, val_dataset, test_dataset, label_encoder = prepare_datasets(
    train_df, val_df, test_df, tokenizer, MAX_LENGTH, strategy="truncate" #head_tail
)

train_loader, val_loader, test_loader = create_data_loaders(
    train_dataset, val_dataset, test_dataset, batch_size=16
)

In [20]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training

In [21]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./output",
    learning_rate=1.1735182865186952e-05,             # Common starting point for BERT
    num_train_epochs=10,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    # per_device_eval_batch_size=32,
    warmup_ratio=0.1,
    weight_decay=0.012916490115700903,
    save_total_limit=10,
    logging_dir="./logs",
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="wandb",
    run_name="mbert_augmented"
)

## Cross entropy  loss

In [22]:
# Get class distribution
class_counts = np.bincount(label_encoder.transform(train_df['annotator_sentiment']))
print("Class distribution:", class_counts)

Class distribution: [7850 7535 7522 7708]


In [23]:
# Calculate balanced weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(label_encoder.transform(train_df['annotator_sentiment'])),
    y=label_encoder.transform(train_df['annotator_sentiment'])
)

In [24]:
mixed_class_index = 0 

In [25]:
class_weights[mixed_class_index]

np.float64(0.975)

In [30]:
# class_weights[mixed_class_index] *= 1.5  # Additional boost

In [26]:
# Convert to tensor
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

In [ ]:
{
    'LABEL_0': 'mixed',
    'LABEL_1': 'negative',
    'LABEL_2': 'neutral',
    'LABEL_3': 'positive',
}

In [31]:
print("Class weights:", class_weights)

Class weights: [0.975      1.01575979 1.01751529 0.99296186]


In [27]:
class WeightedLossTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
        
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """Compatible with all Transformers versions"""
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = inputs.get("labels")
            
        outputs = model(**inputs)
        logits = outputs.logits
        
        if self.class_weights is not None:
            # Ensure weights are on the right device
            weights = self.class_weights.to(logits.device)
            loss_fct = torch.nn.CrossEntropyLoss(weight=weights)
        else:
            loss_fct = torch.nn.CrossEntropyLoss()
            
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

In [28]:
trainer = WeightedLossTrainer(
    class_weights=class_weights_tensor,
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [35]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=6)]
# )

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,1.401700,1.392092,0.261301,0.134260,0.249640,0.261301
100,1.388900,1.377224,0.304024,0.216246,0.372102,0.304024
150,1.372700,1.342104,0.403188,0.364749,0.437582,0.403188
200,1.319300,1.252203,0.503528,0.483366,0.525180,0.503528
250,1.221600,1.135766,0.545597,0.516189,0.565877,0.545597
300,1.146400,1.047032,0.579828,0.553990,0.601144,0.579828
350,0.995700,0.933204,0.626339,0.620028,0.626286,0.626339
400,0.922800,0.890986,0.637053,0.631321,0.658179,0.637053
450,0.874400,0.874373,0.652077,0.635931,0.666202,0.652077


In [30]:
print(123)

123


In [36]:
trainer.save_model()

wandb: 
wandb: 🚀 View run roberta-ukrainian-sentiment at: https://wandb.ai/shynkarov-pn-ukrainian-catholic-university/ukrainian-sentiment/runs/vl9tnzbt
wandb: Find logs at: wandb/run-20250420_080024-vl9tnzbt/logs


# Hyper params tuning

In [28]:
def objective(trial):
    # Define the hyperparameter search space
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1)
    
    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results_optuna",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=2,
        save_total_limit=10,
        # warmup_ratio=0.1,
        num_train_epochs=5,
        weight_decay=weight_decay,
        logging_dir="./logs_optuna",
        logging_steps=50,
        eval_steps=50,
        save_steps=50,
        eval_strategy="steps",
        save_strategy="steps",
        report_to="wandb",
        run_name="optuna",
        metric_for_best_model="f1",
    )

    # Initialize Trainer and train model
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=6)]
    )
    trainer.train()

    # Evaluate the model and return validation accuracy
    eval_results = trainer.evaluate()
    return eval_results["eval_accuracy"]

In [ ]:
# Create and run the Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2025-04-16 23:10:59,094] A new study created in memory with name: no-name-8513712f-27d0-42a3-b86a-4666b4e6c40c
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.917200,0.742903,0.677864,0.674121,0.693244,0.677864
100,0.724700,0.679379,0.707149,0.706546,0.708010,0.707149
150,0.682300,0.679730,0.708010,0.704029,0.725822,0.708010
200,0.697400,0.661064,0.721792,0.718715,0.726476,0.721792
250,0.666200,0.645199,0.732989,0.730710,0.742404,0.732989
300,0.639100,0.663719,0.717485,0.716716,0.723710,0.717485
350,0.569200,0.678832,0.722653,0.723358,0.726377,0.722653
400,0.538900,0.670532,0.721792,0.719441,0.729942,0.721792
450,0.582400,0.656048,0.720930,0.720588,0.722225,0.720930
500,0.537500,0.663014,0.736434,0.734845,0.742807,0.736434


[I 2025-04-16 23:18:29,295] Trial 0 finished with value: 0.7450473729543498 and parameters: {'learning_rate': 1.1735182865186952e-05, 'batch_size': 16, 'weight_decay': 0.012916490115700903}. Best is trial 0 with value: 0.7450473729543498.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.454600,0.820859,0.712317,0.710457,0.728403,0.712317
100,0.390800,0.819037,0.727821,0.727423,0.732244,0.727821
150,0.339600,0.795487,0.716624,0.714898,0.728954,0.716624
200,0.389900,0.819856,0.712317,0.713834,0.720759,0.712317
250,0.374800,0.763311,0.729543,0.729156,0.729498,0.729543


In [30]:
print("Best hyperparameters:", study.best_params)

Best hyperparameters: {'learning_rate': 1.1735182865186952e-05, 'batch_size': 16, 'weight_decay': 0.012916490115700903}


Best hyperparameters: {'learning_rate': 1.1735182865186952e-05, 'batch_size': 16, 'weight_decay': 0.012916490115700903}

In [31]:
study.best_params

{'learning_rate': 1.1735182865186952e-05,
 'batch_size': 16,
 'weight_decay': 0.012916490115700903}

# Classification report

In [31]:
# Get predictions
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)

In [32]:
test_labels = label_encoder.transform(test_df['annotator_sentiment'])

In [33]:
label_encoder.classes_

array(['mixed', 'negative', 'neutral', 'positive'], dtype=object)

In [30]:
# Print classification report
print(classification_report(test_labels, preds))

              precision    recall  f1-score   support

           0       0.60      0.05      0.09        60
           1       0.71      0.75      0.73       455
           2       0.65      0.67      0.66       471
           3       0.65      0.68      0.66       237

    accuracy                           0.67      1223
   macro avg       0.65      0.54      0.53      1223
weighted avg       0.67      0.67      0.66      1223



In [45]:
# Print classification report -- second try
print(classification_report(test_labels, preds))

              precision    recall  f1-score   support

           0       0.32      0.32      0.32        60
           1       0.76      0.70      0.73       455
           2       0.64      0.75      0.69       471
           3       0.72      0.59      0.65       237

    accuracy                           0.68      1223
   macro avg       0.61      0.59      0.59      1223
weighted avg       0.68      0.68      0.68      1223



In [23]:
# Print classification report -- third try
print(classification_report(test_labels, preds))

              precision    recall  f1-score   support

           0       0.33      0.03      0.06        60
           1       0.74      0.76      0.75       455
           2       0.66      0.79      0.72       471
           3       0.76      0.61      0.68       237

    accuracy                           0.71      1223
   macro avg       0.62      0.55      0.55      1223
weighted avg       0.69      0.71      0.69      1223



In [30]:
# Print classification report -- fourth try
print(classification_report(test_labels, preds))

              precision    recall  f1-score   support

           0       0.14      0.02      0.03        60
           1       0.72      0.75      0.74       455
           2       0.68      0.77      0.72       471
           3       0.71      0.64      0.67       237

    accuracy                           0.70      1223
   macro avg       0.56      0.54      0.54      1223
weighted avg       0.68      0.70      0.68      1223



In [30]:
# Print classification report -- fifth try
print(classification_report(test_labels, preds))
# trainer.save_model('best')

              precision    recall  f1-score   support

           0       0.72      0.84      0.77       455
           1       0.72      0.69      0.70       471
           2       0.79      0.63      0.70       237

    accuracy                           0.73      1163
   macro avg       0.74      0.72      0.73      1163
weighted avg       0.74      0.73      0.73      1163



In [ ]:
# Print classification report -- sixth try, mixed class with cross entropy
print(classification_report(test_labels, preds))
# trainer.save_model('best')

In [39]:
# Print classification report -- seventh try, mixed class with cross entropy
print(classification_report(test_labels, preds))
# trainer.save_model('best')

              precision    recall  f1-score   support

           0       0.48      0.20      0.28        60
           1       0.71      0.77      0.74       455
           2       0.69      0.73      0.71       471
           3       0.77      0.67      0.72       237

    accuracy                           0.71      1223
   macro avg       0.66      0.59      0.61      1223
weighted avg       0.70      0.71      0.70      1223



In [35]:
# Print classification report -- mBERT
print(classification_report(test_labels, preds))
# trainer.save_model('best')

              precision    recall  f1-score   support

           0       0.36      0.20      0.26        60
           1       0.73      0.70      0.71       455
           2       0.64      0.77      0.70       471
           3       0.70      0.55      0.62       237

    accuracy                           0.67      1223
   macro avg       0.61      0.56      0.57      1223
weighted avg       0.67      0.67      0.67      1223

